# Atwell - Homework 1 Notebook 4

##Packages, Setup, and Data Import

In [62]:
%matplotlib inline

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split, cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score, explained_variance_score
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, mean_squared_error, median_absolute_error

####Check working directory and change if needed.

In [64]:
%pwd

'C:\\Users\\mlatw\\Desktop\\BIA6303\\Assignment_1'

In [65]:
cd /Users/mlatw/Desktop/BIA6303/Assignment_1

C:\Users\mlatw\Desktop\BIA6303\Assignment_1


Importing the prostate data set and displaying some basic information about it.

In [66]:
pd.options.display.max_rows = 999 #added to display all rows (if needed, but output should be tweaked as required)
prostate = pd.read_csv("prostate.csv") #NA values in this dataset are encoded with a ?
lm_data = prostate #giving it a generic name so this code can be easily reused
print("Data Type", lm_data.dtypes) 
print("Shape of Data", lm_data.shape) #Dimensions of dataset
lm_data.head(5) #See top few rows of dataset

Data Type Obs          int64
lcavol     float64
lweight    float64
age          int64
lbph       float64
svi          int64
lcp        float64
gleason      int64
pgg45        int64
lpsa       float64
dtype: object
Shape of Data (97, 10)


,Obs,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
0,1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783
1,2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519
2,3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519
3,4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519
4,5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564


In [67]:
lm_data.tail(5)

,Obs,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
92,93,2.830268,3.876396,68,-1.386294,1,1.321756,7,60,4.385147
93,94,3.821004,3.896909,44,-1.386294,1,2.169054,7,40,4.684443
94,95,2.907447,3.396185,52,-1.386294,1,2.463853,7,10,5.143124
95,96,2.882564,3.773910,68,1.558145,1,1.558145,7,80,5.477509
96,97,3.471966,3.974998,68,0.438255,1,2.904165,7,20,5.582932


Based on this initial look, all columns are numeric.  Additionally, we see that the 'obs' is a line numer and shouldn't figure in any numerical analysis. 

##Exploratory Data Analysis and Preparation

Now we’ll look at the data to see what changes (if any) need to be made to the dataset.

Now let's list the columns that have null values and how many.

In [68]:
nulls = pd.DataFrame(lm_data.isnull().sum())
nulls.columns = ['Number_of_NaNs']
nulls = nulls[nulls.Number_of_NaNs !=0]
nulls

,Number_of_NaNs


No nulls in the dataset.  Now let's see a summary:

In [69]:
lm_data.describe()

,Obs,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,49.000000,1.350010,3.628943,63.865979,0.100356,0.216495,-0.179366,6.752577,24.381443,2.478387
std,28.145456,1.178625,0.428411,7.445117,1.450807,0.413995,1.398250,0.722134,28.204035,1.154329
min,1.000000,-1.347074,2.374906,41.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,-0.430783
25%,25.000000,0.512824,3.375880,60.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,1.731656
50%,49.000000,1.446919,3.623007,65.000000,0.300105,0.000000,-0.798508,7.000000,15.000000,2.591516
75%,73.000000,2.127041,3.876396,68.000000,1.558145,0.000000,1.178655,7.000000,40.000000,3.056357
max,97.000000,3.821004,4.780383,79.000000,2.326302,1.000000,2.904165,9.000000,100.000000,5.582932


Now we'll drop the column that won't be used in the analysis, in this case 'Obs'.

In [70]:
lm_data_numeric = lm_data.drop(["Obs"], axis=1)
lm_data_numeric.head(5)

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564


No nulls need to be filled, so this code isn't necessary, but executing to maintain consistency with the rest of the code below.

In [71]:
lm_data_numeric_nonull = lm_data_numeric.fillna(lm_data_numeric.median())

Next is code for a correlation matrix, which shows the degree that variables chanhge with each other (1 = perfect positive correlation, 0 = no corrleation, an -1 = perfect negative correlation).

In [72]:
correlations =lm_data_numeric_nonull.corr()
correlations

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
lcavol,1.000000,0.280521,0.225000,0.027350,0.538845,0.675310,0.432417,0.433652,0.734460
lweight,0.280521,1.000000,0.347969,0.442264,0.155385,0.164537,0.056882,0.107354,0.433319
age,0.225000,0.347969,1.000000,0.350186,0.117658,0.127668,0.268892,0.276112,0.169593
lbph,0.027350,0.442264,0.350186,1.000000,-0.085843,-0.006999,0.077820,0.078460,0.179809
svi,0.538845,0.155385,0.117658,-0.085843,1.000000,0.673111,0.320412,0.457648,0.566218
lcp,0.675310,0.164537,0.127668,-0.006999,0.673111,1.000000,0.514830,0.631528,0.548813
gleason,0.432417,0.056882,0.268892,0.077820,0.320412,0.514830,1.000000,0.751905,0.368987
pgg45,0.433652,0.107354,0.276112,0.078460,0.457648,0.631528,0.751905,1.000000,0.422316
lpsa,0.734460,0.433319,0.169593,0.179809,0.566218,0.548813,0.368987,0.422316,1.000000


The strongest correlations with the target variable (lpsa) is lcavol, svi, and lcp. Age and lbph appear to have the weakest correlations.

###Normalize data

Now we'll normalize the data.  All data should be between 0 and 1 after the transformation.

In [73]:
lm_data_numeric_nonull_nm = lm_data_numeric_nonull
lm_data_numeric_nonull_nm = (lm_data_numeric_nonull_nm - lm_data_numeric_nonull_nm.min())/(lm_data_numeric_nonull_nm.max() - lm_data_numeric_nonull_nm.min()) #calculates normalized 0 to 1 value
lm_data_numeric_nonull_nm.describe()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,0.521874,0.521326,0.601736,0.400434,0.216495,0.281305,0.250859,0.243814,0.483756
std,0.228059,0.178098,0.195924,0.390780,0.413995,0.325897,0.240711,0.282040,0.191949
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.359882,0.416123,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.359584
50%,0.540625,0.518858,0.631579,0.454237,0.000000,0.136999,0.333333,0.150000,0.502568
75%,0.672226,0.624196,0.710526,0.793094,0.000000,0.597826,0.333333,0.400000,0.579864
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


We've verified the data is normalized.  Now we’ll designate the target variable and split it off from the dataset.

In [74]:
# designate target variable name and split off from dataset
targetName = 'lpsa'
targetSeries = lm_data_numeric_nonull_nm[targetName]
#remove target from current location and insert in column 0
del lm_data_numeric_nonull_nm[targetName]
#lm_data_numeric_nonull.insert(0, targetName, targetSeries)
#reprint dataframe and see target is in position 0
lm_data_numeric_nonull_nm.head(5)

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
0,0.148460,0.164023,0.236842,0.0,0.0,0.0,0.000000,0.0
1,0.068269,0.392737,0.447368,0.0,0.0,0.0,0.000000,0.0
2,0.161810,0.131507,0.868421,0.0,0.0,0.0,0.333333,0.2
3,0.027689,0.377423,0.447368,0.0,0.0,0.0,0.000000,0.0
4,0.406048,0.439608,0.552632,0.0,0.0,0.0,0.000000,0.0


Now we’re going to split the data into training and test sets, in a 75/25 ratio.  

In [75]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(lm_data_numeric_nonull_nm, targetSeries, test_size=0.25, random_state=33)

The following is code to look at min, max, and mean of the training and test sets.  We won't execute it here, but I'm leaving it in for future use.

In [76]:
#print(np.max(X_train), np.min(X_train), np.mean(X_train), np.max(y_train), np.min(y_train), np.mean(y_train))

##Running linear models

Now that the dataset is prepared, we can run the different linear models.

###Ridge Regression

The first linear model we'll use is the Ridge Regression.  As the Ridge has a peanlty incorporated (alpha), we'll start with the default value and then use a grid search to find the optimum value.  In the case of the Ridge Regression, it uses 'L2' for its penalty (alpha), which penalizes based on the square of the weight.  It tends to result in fewer factors reduced to zero than 'L1', which uses the absolute value of the weight.

In [77]:
# Ridge Regression
from sklearn.linear_model import Ridge
model_rg = Ridge() #ridge with default alpha
model_rg.fit(X_train, y_train)
print(model_rg) #shows parameters and their default values
expected_rg = y_train
predicted_rg = model_rg.predict(X_train)
print("Ridge Model Coefficients:")
print(model_rg.coef_)
print("Mean Squared Error:")
print(mean_squared_error(expected_rg,predicted_rg))
print("R-Squared:")
print(r2_score(expected_rg,predicted_rg))
print("Explained Variance Score:")
print(explained_variance_score(expected_rg,predicted_rg))
print("Median Absolute Error:") 
print(median_absolute_error(expected_rg,predicted_rg))

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
Ridge Model Coefficients:
[ 0.32731261  0.15531845 -0.05028167  0.10058787  0.17424164  0.01772567
  0.03317449  0.06574799]
Mean Squared Error:
0.00859069788912
R-Squared:
0.753592211425
Explained Variance Score:
0.753592211425
Median Absolute Error:
0.0527039053157


With the default value of alpha = 1, the mean squared error in this case was 0.00859, the median absolute error 0.0527, and the R-squared was 0.754.  None of the parameters was reduced to zero.  Now let's do the grid search and see what the optimum value of alpha is.

In [78]:
# use a full grid over several parameters and cross validate 5 times
from sklearn.model_selection import GridSearchCV
param_grid = {"alpha": [.01,.05,.1,.5, 1, 2,3,4,5,6,7,8,9,10,15,20]}
# run grid search
grid_search = GridSearchCV(model_rg, param_grid=param_grid,n_jobs=-1,cv=5,return_train_score=True)
grid_search.fit(X_train, y_train)
print("Best", grid_search.best_params_) 
print("Grid Scores:")
pd.DataFrame(grid_search.cv_results_)

Best {'alpha': 0.5}
Grid Scores:


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.043746,0.003126,0.639881,0.787519,0.01,{'alpha': 0.01},5,0.743137,0.781925,0.603231,...,0.853175,0.755693,0.750763,0.775470,0.244342,0.826911,0.025003,0.006252,0.210140,0.023844
1,0.006250,0.003125,0.643034,0.787232,0.05,{'alpha': 0.05},3,0.743019,0.781584,0.615152,...,0.844560,0.755470,0.756771,0.775156,0.250518,0.826706,0.007655,0.006250,0.206433,0.023854
2,0.015624,0.000000,0.645995,0.786438,0.1,{'alpha': 0.1},2,0.742351,0.780642,0.627211,...,0.833950,0.754862,0.763169,0.774275,0.257754,0.826128,0.024206,0.000000,0.202127,0.023878
3,0.006250,0.003125,0.650720,0.772327,0.5,{'alpha': 0.5},1,0.727337,0.763925,0.665886,...,0.761842,0.744650,0.787627,0.758034,0.304351,0.814839,0.007654,0.006250,0.175055,0.024125
4,0.009377,0.000000,0.642642,0.751078,1,{'alpha': 1},4,0.703032,0.738967,0.665105,...,0.699622,0.729743,0.789490,0.733141,0.350044,0.796046,0.007656,0.000000,0.149453,0.024441
5,0.003124,0.000000,0.622714,0.713961,2,{'alpha': 2},6,0.658374,0.696203,0.636198,...,0.622411,0.703175,0.769750,0.690333,0.423329,0.759952,0.006249,0.000000,0.110619,0.025073
6,0.006250,0.000000,0.604523,0.683410,3,{'alpha': 3},7,0.620485,0.661984,0.604914,...,0.573365,0.680118,0.743766,0.656240,0.478914,0.727885,0.007655,0.000000,0.084141,0.025460
7,0.012500,0.000000,0.587722,0.656947,4,{'alpha': 4},8,0.587584,0.633102,0.576828,...,0.537249,0.659142,0.717579,0.627533,0.520160,0.698866,0.018221,0.000000,0.068458,0.025610
8,0.003124,0.000000,0.571748,0.633203,5,{'alpha': 5},9,0.558398,0.607742,0.552002,...,0.508304,0.639571,0.692384,0.602335,0.550016,0.672167,0.006248,0.000000,0.061827,0.025616
9,0.006251,0.003126,0.556346,0.611456,6,{'alpha': 6},10,0.532132,0.584920,0.529890,...,0.483879,0.621091,0.668455,0.579647,0.570991,0.647374,0.007656,0.006252,0.061438,0.025544


The grid search data computes alpha=0.5 as the best result.  Let's put that into the Ridge model and see the result.

In [79]:
# Ridge Regression
from sklearn.linear_model import Ridge
model_rg_al = Ridge(alpha=0.5)
model_rg_al.fit(X_train, y_train)
print(model_rg_al) #shows parameters and their default values
expected_rg = y_train
predicted_rg = model_rg_al.predict(X_train)
print("Ridge Model Coefficients:")
print(model_rg_al.coef_)
print("Mean Squared Error:")
print(mean_squared_error(expected_rg,predicted_rg))
print("R-Squared:")
print(r2_score(expected_rg,predicted_rg))
print("Explained Variance Score:")
print(explained_variance_score(expected_rg,predicted_rg))
print("Median Absolute Error:") 
print(median_absolute_error(expected_rg,predicted_rg))

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
Ridge Model Coefficients:
[ 0.38386064  0.18404992 -0.07692148  0.10423687  0.1784481  -0.01365929
  0.02661234  0.07787221]
Mean Squared Error:
0.00801957992442
R-Squared:
0.769973641259
Explained Variance Score:
0.769973641259
Median Absolute Error:
0.0536551782714


The resulting Ridge model has a mean squared error of 0.00802, a median absolute error of 0.0536, and the R-squared was 0.770, which is slightly more accurate than the default setting of alpha=1.  Again, the alpha didn't reduce any of the factors to zero, so this model still has 8 variables. 

###LASSO Regression Model

Now we'll try the LASSO Regression, which like the Ridge has a penalty incorporated (alpha), so we'll start with the default value and then use a grid search to find the optimum value.  Unlike the Ridge Regression, it uses 'L1' for its penalty (alpha), which penalizes based on the absolute value of the weight, typically resulting in more factors being reduced to zero.  Thus, it can be useful to reduce a large number of dimensions.

In [80]:
# Lasso Regression
# fit a LASSO model to the data
from sklearn.linear_model import Lasso
model_ls = Lasso() #LASSO with default vaues
model_ls.fit(X_train, y_train)
print(model_ls) #shows model parameters
# make predictions
expected_ls = y_train
predicted_ls = model_ls.predict(X_train)
# summarize the fit of the model
print("LASSO Model Coefficients:")
print(model_ls.coef_)
print("Mean Squared Error:")
print(mean_squared_error(expected_ls,predicted_ls))
print("R-Squared:")
print(r2_score(expected_ls,predicted_ls))
print("Explained Variance Score:")
print(explained_variance_score(expected_ls,predicted_ls))
print("Median Absolute Error:") 
print(median_absolute_error(expected_ls,predicted_ls))

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
LASSO Model Coefficients:
[ 0.  0.  0.  0.  0.  0.  0.  0.]
Mean Squared Error:
0.0348637433045
R-Squared:
0.0
Explained Variance Score:
2.22044604925e-16
Median Absolute Error:
0.113175264987


The default setting for the alpha (1) reduced all of the factors to zero!  The mean squared error in this case was 0.0348 and the R-squared was 0.0, but those don't have any effective meaning here.   Now let's do the grid search and see what the optimum value of alpha is.

In [81]:
# use a full grid over several parameters and cross validate 5 times
from sklearn.model_selection import GridSearchCV
param_grid = {"alpha": [.000001, .00001, .0001,.001,.01,.02, .05, .1, .2, .3,.4,.5,.6,.7,.8,.9,1]}
# run grid search
grid_search = GridSearchCV(model_ls, param_grid=param_grid,n_jobs=-1,cv=5,return_train_score=True)
grid_search.fit(X_train, y_train)
print("Best", grid_search.best_params_) 
print("Grid Scores:")
pd.DataFrame(grid_search.cv_results_)

Best {'alpha': 0.001}
Grid Scores:


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.012500,0.000000,0.638987,0.787532,1e-06,{'alpha': 1e-06},4,0.743111,0.781940,0.599974,...,0.855276,0.755704,0.749139,0.775484,0.242785,0.826920,0.006250,0.000000,0.211079,0.023843
1,0.009376,0.000000,0.639296,0.787531,1e-05,{'alpha': 1e-05},3,0.743213,0.781939,0.600938,...,0.854734,0.755703,0.749291,0.775483,0.243622,0.826919,0.007655,0.000000,0.210655,0.023843
2,0.003125,0.000000,0.642287,0.787424,0.0001,{'alpha': 0.0001},2,0.744130,0.781804,0.610674,...,0.849054,0.755594,0.750742,0.775397,0.251820,0.826837,0.006251,0.000000,0.206434,0.023850
3,0.003125,0.003124,0.657714,0.778411,0.001,{'alpha': 0.001},1,0.740214,0.769695,0.660693,...,0.802968,0.749679,0.761192,0.766751,0.317395,0.818622,0.006249,0.006248,0.173564,0.023382
4,0.003125,0.000000,0.578790,0.660786,0.01,{'alpha': 0.01},5,0.591794,0.629251,0.518201,...,0.580317,0.652739,0.686740,0.634863,0.520297,0.709655,0.006251,0.000000,0.061087,0.029652
5,0.006250,0.000000,0.352107,0.428180,0.02,{'alpha': 0.02},6,0.307680,0.339448,0.276227,...,0.273500,0.419538,0.448575,0.396148,0.463149,0.495804,0.007655,0.000000,0.083781,0.058931
6,0.006250,0.000000,-0.021704,0.000000,0.05,{'alpha': 0.05},7,-0.047905,0.000000,-0.005272,...,-0.039230,0.000000,-0.002761,0.000000,-0.012653,0.000000,0.007655,0.000000,0.018566,0.000000
7,0.003125,0.003126,-0.021704,0.000000,0.1,{'alpha': 0.1},7,-0.047905,0.000000,-0.005272,...,-0.039230,0.000000,-0.002761,0.000000,-0.012653,0.000000,0.006250,0.006252,0.018566,0.000000
8,0.006252,0.000000,-0.021704,0.000000,0.2,{'alpha': 0.2},7,-0.047905,0.000000,-0.005272,...,-0.039230,0.000000,-0.002761,0.000000,-0.012653,0.000000,0.007657,0.000000,0.018566,0.000000
9,0.009375,0.000000,-0.021704,0.000000,0.3,{'alpha': 0.3},7,-0.047905,0.000000,-0.005272,...,-0.039230,0.000000,-0.002761,0.000000,-0.012653,0.000000,0.007655,0.000000,0.018566,0.000000


I had to use much lower values of alpha in this case when hunting for the optimal solution (not surprisingly, as we saw an alpha of one reducing all terms to zero).  The grid search determined that alpha=0.001 was best.  Let's put that in the LASSO model and see the result.

In [82]:
# Lasso Regression with alpha value determined by grid search
# fit a LASSO model to the data
from sklearn.linear_model import Lasso
model_ls_al = Lasso(alpha=0.001) 
model_ls_al.fit(X_train, y_train)
print(model_ls_al) #shows model parameters
# make predictions
expected_ls = y_train
predicted_ls = model_ls_al.predict(X_train)
# summarize the fit of the model
print("LASSO Model Coefficients:")
print(model_ls_al.coef_)
print("Mean Squared Error:")
print(mean_squared_error(expected_ls,predicted_ls))
print("R-Squared:")
print(r2_score(expected_ls,predicted_ls))
print("Explained Variance Score:")
print(explained_variance_score(expected_ls,predicted_ls))
print("Median Absolute Error:") 
print(median_absolute_error(expected_ls,predicted_ls))

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
LASSO Model Coefficients:
[ 0.43167195  0.17574678 -0.05943902  0.09817497  0.17002846 -0.00093135
  0.          0.06759174]
Mean Squared Error:
0.00797137786088
R-Squared:
0.771356225542
Explained Variance Score:
0.771356225542
Median Absolute Error:
0.0558427969484


The resulting LASSO model has a mean squared error of 0.00797, a median absolute error of 0.0558, and a R-squared of 0.771, on par with the Ridge results.  Only one term were penalized to zero.

###Elastic Net Regression

Now we'll try the Elasitc Net Regression, which also has a penalty incorporated (alpha), so we'll start with the default value and then use a grid search to find the optimum value.  The Elasitc Net uses a combination of L1 and L2 for its penalty. 

In [83]:
# ElasticNet Regression
# fit a model to the data
from sklearn.linear_model import ElasticNet
model_en = ElasticNet()
model_en.fit(X_train, y_train)
print(model_en)
# make predictions
expected_en = y_train
predicted_en = model_en.predict(X_train)
# summarize the fit of the model
print("Elastic Net Model Coefficients:")
print(model_en.coef_)
print("Mean Squared Error:")
print(mean_squared_error(expected_en,predicted_en))
print("R-Squared:")
print(r2_score(expected_en,predicted_en))
print("Explained Variance Score:")
print(explained_variance_score(expected_en,predicted_en))
print("Median Absolute Error:") 
print(median_absolute_error(expected_en,predicted_en))

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
Elastic Net Model Coefficients:
[ 0.  0.  0.  0.  0.  0.  0.  0.]
Mean Squared Error:
0.0348637433045
R-Squared:
0.0
Explained Variance Score:
2.22044604925e-16
Median Absolute Error:
0.113175264987


Similar to the LASSO, the default setting for the alpha (1) reduced all of the factors to zero!  The mean squared error in this case was 0.0349 and the R-squared was 0.0, but those don't have any effective meaning here.   Now let's do the grid search and see what the optimum value of alpha is.

In [84]:
# use a full grid over several parameters and cross validate 5 times
from sklearn.model_selection import GridSearchCV
param_grid = {"alpha": [.000001, .00001, .0001,.001,.01,.02, .05, .1, .2, .3,.4,.5,.6,.7,.8,.9,1]}
# run grid search
grid_search = GridSearchCV(model_en, param_grid=param_grid,n_jobs=-1,cv=5,return_train_score=True)
grid_search.fit(X_train, y_train)
print("Best", grid_search.best_params_) 
print("Grid Scores:")
pd.DataFrame(grid_search.cv_results_)

Best {'alpha': 0.001}
Grid Scores:


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.021874,0.006249,0.638983,0.787532,1e-06,{'alpha': 1e-06},4,0.743104,0.781940,0.599941,...,0.855300,0.755704,0.749135,0.775484,0.242788,0.826920,0.021196,0.007654,0.211083,0.023843
1,0.003126,0.003126,0.639146,0.787531,1e-05,{'alpha': 1e-05},3,0.743158,0.781940,0.600473,...,0.854973,0.755703,0.749255,0.775484,0.243204,0.826919,0.006251,0.006251,0.210863,0.023843
2,0.015623,0.003125,0.640912,0.787495,0.0001,{'alpha': 0.0001},2,0.743645,0.781894,0.606258,...,0.851624,0.755667,0.750418,0.775453,0.247753,0.826892,0.024204,0.006250,0.208494,0.023845
3,0.000000,0.006249,0.653632,0.784178,0.001,{'alpha': 0.001},1,0.745221,0.777494,0.649743,...,0.819355,0.753104,0.759931,0.772500,0.287646,0.824220,0.000000,0.007654,0.187971,0.023825
4,0.009374,0.000000,0.630272,0.721911,0.01,{'alpha': 0.01},5,0.668623,0.704952,0.603267,...,0.643597,0.701876,0.762569,0.703579,0.472494,0.765310,0.007654,0.000000,0.093470,0.024701
5,0.009376,0.000000,0.546246,0.627126,0.02,{'alpha': 0.02},6,0.547339,0.589715,0.480623,...,0.519323,0.624531,0.665045,0.595329,0.523507,0.678280,0.007655,0.000000,0.062315,0.033047
6,0.006251,0.000000,0.180779,0.249695,0.05,{'alpha': 0.05},7,0.107558,0.137188,0.136015,...,0.091459,0.260938,0.254237,0.200980,0.323053,0.316187,0.007655,0.000000,0.089838,0.072868
7,0.006250,0.000000,-0.021704,0.000000,0.1,{'alpha': 0.1},8,-0.047905,0.000000,-0.005272,...,-0.039230,0.000000,-0.002761,0.000000,-0.012653,0.000000,0.007655,0.000000,0.018566,0.000000
8,0.000000,0.006249,-0.021704,0.000000,0.2,{'alpha': 0.2},8,-0.047905,0.000000,-0.005272,...,-0.039230,0.000000,-0.002761,0.000000,-0.012653,0.000000,0.000000,0.007654,0.018566,0.000000
9,0.009376,0.000000,-0.021704,0.000000,0.3,{'alpha': 0.3},8,-0.047905,0.000000,-0.005272,...,-0.039230,0.000000,-0.002761,0.000000,-0.012653,0.000000,0.007656,0.000000,0.018566,0.000000


Similar to LASSO, I had to use much lower values of alpha in this case when hunting for the optimal solution (not surprisingly, as we saw an alpha of one reducing all terms to zero).  The grid search determined that alpha=0.001 was best.  Let's put that in the Elastic Net model and see the result.

In [85]:
# ElasticNet Regression
# fit a model to the data
from sklearn.linear_model import ElasticNet
model_en_al = ElasticNet(alpha=0.001)
model_en_al.fit(X_train, y_train)
print(model_en_al)
# make predictions
expected_en = y_train
predicted_en = model_en_al.predict(X_train)
# summarize the fit of the model
print("Elastic Net Model Coefficients:")
print(model_en_al.coef_)
print("Mean Squared Error:")
print(mean_squared_error(expected_en,predicted_en))
print("R-Squared:")
print(r2_score(expected_en,predicted_en))
print("Explained Variance Score:")
print(explained_variance_score(expected_en,predicted_en))
print("Median Absolute Error:") 
print(median_absolute_error(expected_en,predicted_en))

ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
Elastic Net Model Coefficients:
[  4.47467325e-01   1.99466961e-01  -8.85351556e-02   1.02795939e-01
   1.75494079e-01  -3.23424897e-02   1.61504668e-04   8.94991416e-02]
Mean Squared Error:
0.0077638444699
R-Squared:
0.777308925146
Explained Variance Score:
0.777308925146
Median Absolute Error:
0.0514926334665


The resulting Elastic Net model has a mean squared error of 0.00776, a median absolute error of 0.0515, and a R-squared of 0.777, slightly better than the Ridge and LASSO results.  No terms were penalized to zero.

###LARS (Least Angle Regression) model

Now we'll look at the LARS, which conducts the regression in steps by finding the column most closely correlated with the target. 

In [86]:
#LARS Regression Model- Least Angle Regression model
from sklearn import linear_model
model_LAR = linear_model.Lars() #default numer of n_nonzero_coefs
model_LAR.fit(X_train, y_train)
print(model_LAR)
# make predictions
expected_LAR = y_train
predicted_LAR = model_LAR.predict(X_train)
# summarize the fit of the model
#mse_LAR = np.mean((predicted_LAR-expected_LAR)**2)
print("LARS intercept:")
print(model_LAR.intercept_)
print("LARS Coefficients:")
print(model_LAR.coef_)
print("Mean Squared Error:")
print(mean_squared_error(expected_LAR,predicted_LAR))
print("R-Squared:")
print(r2_score(expected_LAR,predicted_LAR))
print("Explained Variance Score:")
print(explained_variance_score(expected_LAR,predicted_LAR))
print("Median Absolute Error:") 
print(median_absolute_error(expected_LAR,predicted_LAR))

Lars(copy_X=True, eps=2.2204460492503131e-16, fit_intercept=True,
   fit_path=True, n_nonzero_coefs=500, normalize=True, positive=False,
   precompute='auto', verbose=False)
LARS intercept:
0.0981914852983
LARS Coefficients:
[ 0.47606464  0.23312088 -0.12713002  0.10808687  0.18188083 -0.07502736
  0.0105249   0.10966399]
Mean Squared Error:
0.00765717158932
R-Squared:
0.780368633327
Explained Variance Score:
0.780368633327
Median Absolute Error:
0.0621096254695


The LARS produce a mean squared error of 0.00766, a median absolute error of 0.0621, and a R squared of 0.78.  The default number of remaining values (the paremeter is called "n nonzero coefficients") is 500 in the LARS, so the model used all 8 parameters.  Let's use a grid serach to find the optimal number.

In [87]:
# use a full grid over several parameters and cross validate 5 times
from sklearn.model_selection import GridSearchCV
param_grid = {"n_nonzero_coefs": [1,2,3,4,5,6,7,8,]}
#param_grid={"alpha": [0,10,1]} #this does a range 1 through 10 changes by a factor of 1. 
#param_grid={"alpha": [.01,1,.05]} #this does a range 1 through 1 changes by a factor of .05

# run grid search
grid_search = GridSearchCV(model_LAR, param_grid=param_grid,n_jobs=-1,cv=5,return_train_score=True)
grid_search.fit(X_train, y_train)
print("Best", grid_search.best_params_) 
print("Grid Scores:")
pd.DataFrame(grid_search.cv_results_)

Best {'n_nonzero_coefs': 6}
Grid Scores:


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_n_nonzero_coefs,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.084377,0.000000,0.318160,0.343187,1,{'n_nonzero_coefs': 1},8,0.338821,0.426984,0.291814,...,0.243542,0.202645,0.409598,0.483104,0.307431,0.305332,0.071673,0.000000,0.054445,0.099837
1,0.006251,0.000000,0.468678,0.516809,2,{'n_nonzero_coefs': 2},7,0.396901,0.475411,0.443795,...,0.535367,0.507454,0.451454,0.516573,0.522779,0.528935,0.007656,0.000000,0.051903,0.026301
2,0.009375,0.006250,0.594725,0.649047,3,{'n_nonzero_coefs': 3},6,0.552042,0.611894,0.550099,...,0.646112,0.598344,0.624683,0.659918,0.606928,0.657574,0.007655,0.012500,0.038876,0.042020
3,0.006251,0.000000,0.615940,0.684403,4,{'n_nonzero_coefs': 4},5,0.577946,0.636387,0.555011,...,0.715054,0.672000,0.641128,0.671313,0.597628,0.720835,0.007655,0.000000,0.056328,0.032658
4,0.018752,0.003125,0.666325,0.758333,5,{'n_nonzero_coefs': 5},2,0.714050,0.751312,0.633416,...,0.788818,0.734298,0.745440,0.738677,0.448841,0.799491,0.022966,0.006250,0.118456,0.023658
5,0.003125,0.006251,0.678000,0.768025,6,{'n_nonzero_coefs': 6},1,0.738737,0.769889,0.657851,...,0.797412,0.743556,0.749136,0.747032,0.443967,0.800169,0.006250,0.012501,0.123530,0.021011
6,0.015625,0.000000,0.660952,0.782385,7,{'n_nonzero_coefs': 7},3,0.745935,0.777044,0.644222,...,0.832077,0.754187,0.750283,0.766990,0.327371,0.819284,0.017117,0.000000,0.174487,0.022658
7,0.012500,0.000000,0.638962,0.787532,8,{'n_nonzero_coefs': 8},4,0.743097,0.781940,0.599874,...,0.855335,0.755704,0.749122,0.775484,0.242736,0.826920,0.011692,0.000000,0.211110,0.023843


The grid serach determined that 6 nonzero coefficients is the best answer.  Let's put that number into the LARS.

In [88]:
#LARS Regression Model- Least Angle Regression model
from sklearn import linear_model
model_LAR_cf = linear_model.Lars(n_nonzero_coefs=6)
model_LAR_cf.fit(X_train, y_train)
print(model_LAR_cf)
# make predictions
expected_LAR = y_train
predicted_LAR = model_LAR_cf.predict(X_train)
# summarize the fit of the model
print("LARS intercept:")
print(model_LAR_cf.intercept_)
print("LARS Coefficients:")
print(model_LAR_cf.coef_)
print("Mean Squared Error:")
print(mean_squared_error(expected_LAR,predicted_LAR))
print("R-Squared:")
print(r2_score(expected_LAR,predicted_LAR))
print("Explained Variance Score:")
print(explained_variance_score(expected_LAR,predicted_LAR))
print("Median Absolute Error:") 
print(median_absolute_error(expected_LAR,predicted_LAR))

Lars(copy_X=True, eps=2.2204460492503131e-16, fit_intercept=True,
   fit_path=True, n_nonzero_coefs=6, normalize=True, positive=False,
   precompute='auto', verbose=False)
LARS intercept:
0.104007351457
LARS Coefficients:
[ 0.43500287  0.2037187  -0.06765826  0.09094705  0.1632417   0.          0.
  0.06788898]
Mean Squared Error:
0.00794167265944
R-Squared:
0.772208262605
Explained Variance Score:
0.772208262605
Median Absolute Error:
0.0523219153454


The LARS has a mean squared error of 0.0079, a median absolute error of 0.0523, and an R-squared of 0.772.  R Squared and MSE were slightly worse with the new number of coefficients, median absolute error slighly better.  Overall, these results are on par with the other results seen so far.    

###Cross validation

Now we're going to conduct a cross validation on the Elastic net model with the alpha=0.001 as found during the grid search.    Cross validation is a process where we take a subset of the data (in this case, 33% of it) and a this data is compared to a model generated by the remaining data (in this case, 67% of it).  This process is repeated for a total of 3 times (in our example), with each run using a different 33% of the data.  I used three fold cross validation as this is a very small dataset and larger numbers would produce wide variation in results.   

In [89]:
#verify Ridge with Cross Validation
scores = cross_val_score(model_en_al, X_train, y_train, cv=3)
print("Cross Validation Score for each K",scores)
print("Mean Cross Validation Score", scores.mean()) 

Cross Validation Score for each K [ 0.69964072  0.75578668  0.28330527]
Mean Cross Validation Score 0.579577558105


The cross validation revealed a wide range of scores from 0.284 to 0.756.  This suggests the model is overfit, but the small size of the dataset may also be playing a role.  Since the LASSO results were similar, we'll run a Cross Validation on the LASSO as well:

In [90]:
#verify LASSO with Cross Validation
scores = cross_val_score(model_ls_al, X_train, y_train, cv=3)
print("Cross Validation Score for each K",scores)
print("Mean Cross Validation Score", scores.mean())

Cross Validation Score for each K [ 0.70223651  0.76106091  0.29958882]
Mean Cross Validation Score 0.587628748631


The LASSO cross validation results also show a significant spread, from 0.3 to 0.761, about the same as the Elastic Net.

##Conclusion

Based on the Mean Squared Error, Median Absolute Error, and R Squared scores, the four models, when using the alpha or coefficient number as determined by their grid searches, performed very similarly. The Elastic Net model had was slightly better than the rest in that it had the lowest median absolute error, the highest R-squared, and the lowest MSE; but, the differences were slight.  I would use the Elastic Net due to these factors but the small size of the dataset prevents making an authoritative statement.  